Импортируем все нужные нам функции из пакетов pandas, scikit-learn и time

In [53]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import precision_score, recall_score, accuracy_score, classification_report, confusion_matrix, accuracy_score, confusion_matrix 
from sklearn.pipeline import Pipeline

In [2]:
from sklearn.svm import SVC, LinearSVC

In [3]:
from time import clock

Создаем функцию pipeline_fit, которой передаем параметры corpus (объект нашего корпуса), is_binary (использователь бинарное или частотное представление), ngram_range (какие n-граммы использовать)

In [64]:
def pipeline_fit(corpus, is_binary=False, ngram_range=(1,1), is_linear=True):
    norm = None
    if is_linear:
        norm = u'l2'
    pipeline = Pipeline([
        ('vectorizer', TfidfVectorizer(use_idf=is_linear, norm=norm, analyzer="word", ngram_range=ngram_range, max_features=100000000, binary=is_binary)),
        ('clf', LinearSVC())
    ]) # 
    pipeline.fit(corpus.data, corpus.lang) # обучаем
    return pipeline

Сделаем отдельную функцию, которая принимает в качестве аргументов название папки и файла и возвращает содержимое этого файла 

In [48]:
def filename_to_text(folder, filename):
    PATH = folder + "/tokenized/"
    f = open(PATH + filename, "r")
    return " ".join([l[:-1] for l in f])

Читаем файл с корпусом

In [49]:
corpus_sample_train = pd.read_csv("train/index-training.csv", header=None, names=("filename", "type", "lang", "level"))

Поскольку корпус индексированный, то есть в столбце filename лежат названия файлов с текстами, а не сами тексты, нам нужно сделать столбец именно с текстами. Делаем это так: к каждой ячейке столбца filename применяем функцию filename_to_text, обернув ее в лямбду просто для того, чтобы мы могли передать этой функции папку с текстами (типа просто забиндили аргументы)

In [50]:
corpus_sample_train['data'] = corpus_sample_train['filename'].apply(lambda x : filename_to_text("train", x))

Просто массивчики с вариантами конфигурации наших моделек

In [65]:
binary_variants = [True, False]
linear_variants = [True, False]
ngram_variants = [(1,1), (2,2), (1,2)]

Для каждой комбинации вариантов обучим модель и добавим результат обучения в массив res, заодно замеряем и выводим время работы.

In [67]:
res = []
for bv in binary_variants:
    for lv in linear_variants:
        for nv in ngram_variants:
            start_time = clock()
            res.append((bv, nv, lv, pipeline_fit(corpus_sample_train, bv, nv, lv)))
            print("\n\nConfiguration: binary=" + str(bv) +"; ngram_range=(" + (",".join([str(x) for x in nv])) + "), linear=" + str(lv) + "\n")
            print(str(clock() - start_time) + "s elapsed\n")



Configuration: binary=True; ngram_range=(1,1), linear=True

2.76016801986s elapsed



Configuration: binary=True; ngram_range=(2,2), linear=True

8.34278132099s elapsed



Configuration: binary=True; ngram_range=(1,2), linear=True

11.1113102223s elapsed



Configuration: binary=True; ngram_range=(1,1), linear=False

3.71544388296s elapsed



Configuration: binary=True; ngram_range=(2,2), linear=False

8.77536987799s elapsed



Configuration: binary=True; ngram_range=(1,2), linear=False

11.9964737392s elapsed



Configuration: binary=False; ngram_range=(1,1), linear=True

2.86539990051s elapsed



Configuration: binary=False; ngram_range=(2,2), linear=True

8.30669322732s elapsed



Configuration: binary=False; ngram_range=(1,2), linear=True

11.3656639485s elapsed



Configuration: binary=False; ngram_range=(1,1), linear=False

11.5130949671s elapsed



Configuration: binary=False; ngram_range=(2,2), linear=False

9.89123138347s elapsed



Configuration: binary=False; ngram_range=(

Читаем тестовые данные

In [68]:
corpus_sample_test = pd.read_csv("test/index-dev.csv", header=None, names=("filename", "type", "lang", "level"))

Как и в случае с корпусом, на котором мы обучали модель, тестовые данные индексированные. Делаем то же самое, но, разумеется, изменяем папку с текстовыми файлами в лямбда-функции.

In [69]:
corpus_sample_test['data'] = corpus_sample_test['filename'].apply(lambda x : filename_to_text("test", x))

Предсказываем результаты на тестовых данных и выводим precision и accuracy для каждой из моделей.

In [70]:
for result in res:
    
    predictions = result[3].predict(corpus_sample_test.data)
    print("\n\nConfiguration: binary=" + str(result[0]) +"; ngram_range=(" + (",".join([str(x) for x in result[1]])) + "), linear=" + str(result[2]) + "\n")
    print(classification_report(corpus_sample_test.lang, predictions))
    print("\nAccuracy: " + str(accuracy_score(corpus_sample_test.lang, predictions)))
    print(confusion_matrix(corpus_sample_test.lang, predictions))



Configuration: binary=True; ngram_range=(1,1), linear=True

             precision    recall  f1-score   support

        ARA       0.84      0.77      0.80       100
        CHI       0.70      0.74      0.72       100
        FRE       0.81      0.85      0.83       100
        GER       0.78      0.91      0.84       100
        HIN       0.64      0.69      0.66       100
        ITA       0.87      0.85      0.86       100
        JPN       0.72      0.69      0.70       100
        KOR       0.77      0.70      0.73       100
        SPA       0.81      0.71      0.76       100
        TEL       0.69      0.71      0.70       100
        TUR       0.81      0.79      0.80       100

avg / total       0.77      0.76      0.76      1100


Accuracy: 0.764545454545
[[77  3  3  2  4  0  2  1  2  3  3]
 [ 1 74  1  0  2  0  6  8  0  2  6]
 [ 0  2 85  6  2  1  2  0  2  0  0]
 [ 0  0  3 91  0  2  0  1  3  0  0]
 [ 1  2  1  1 69  0  0  1  1 23  1]
 [ 0  1  4  3  0 85  0  0  3  1  3]
 [ 4

Сейчас мы проделаем то же самое, но добавим функцию, которая будет убирать знаки препинания из текстов, и обучим и протестируем наши модели уже на текстах без них.

In [71]:
def remove_punctuation(s):
    marks = set("!\"#$%&')*+,-./:;<=>?@[\]^_`{|}~")
    marks.add("--")
    return " ".join([word for word in s.split() if word not in marks])

In [72]:
corpus_sample_train['data'] = corpus_sample_train['filename'].apply(lambda x : remove_punctuation(filename_to_text("train", x)))

In [73]:
corpus_sample_test['data'] = corpus_sample_test['filename'].apply(lambda x : remove_punctuation(filename_to_text("test", x)))

In [75]:
res_no_punc = []
for bv in binary_variants:
    for lv in linear_variants:
        for nv in ngram_variants:
            start_time = clock()
            res_no_punc.append((bv, nv, lv, pipeline_fit(corpus_sample_train, bv, nv, lv)))
            print("\n\nConfiguration: binary=" + str(bv) +"; ngram_range=(" + (",".join([str(x) for x in nv])) + "), linear=" + str(lv) + "\n")
            print(str(clock() - start_time) + "s elapsed\n")



Configuration: binary=True; ngram_range=(1,1), linear=True

2.80748741524s elapsed



Configuration: binary=True; ngram_range=(2,2), linear=True

8.50427829306s elapsed



Configuration: binary=True; ngram_range=(1,2), linear=True

11.6657336204s elapsed



Configuration: binary=True; ngram_range=(1,1), linear=False

3.76195669919s elapsed



Configuration: binary=True; ngram_range=(2,2), linear=False

9.25020075058s elapsed



Configuration: binary=True; ngram_range=(1,2), linear=False

12.4442168888s elapsed



Configuration: binary=False; ngram_range=(1,1), linear=True

3.00805889331s elapsed



Configuration: binary=False; ngram_range=(2,2), linear=True

8.79339182501s elapsed



Configuration: binary=False; ngram_range=(1,2), linear=True

11.4562856838s elapsed



Configuration: binary=False; ngram_range=(1,1), linear=False

10.7722087111s elapsed



Configuration: binary=False; ngram_range=(2,2), linear=False

9.57161960429s elapsed



Configuration: binary=False; ngram_range=(

In [76]:
for result in res_no_punc:
    
    predictions = result[3].predict(corpus_sample_test.data)
    print("\n\nConfiguration: binary=" + str(result[0]) +"; ngram_range=(" + (",".join([str(x) for x in result[1]])) + "), linear=" + str(result[2]) + "\n")
    print(classification_report(corpus_sample_test.lang, predictions))
    print("\nAccuracy: " + str(accuracy_score(corpus_sample_test.lang, predictions)))
    print(confusion_matrix(corpus_sample_test.lang, predictions))



Configuration: binary=True; ngram_range=(1,1), linear=True

             precision    recall  f1-score   support

        ARA       0.84      0.77      0.80       100
        CHI       0.70      0.74      0.72       100
        FRE       0.81      0.85      0.83       100
        GER       0.78      0.91      0.84       100
        HIN       0.64      0.69      0.66       100
        ITA       0.87      0.85      0.86       100
        JPN       0.72      0.69      0.70       100
        KOR       0.77      0.70      0.73       100
        SPA       0.81      0.71      0.76       100
        TEL       0.69      0.71      0.70       100
        TUR       0.81      0.79      0.80       100

avg / total       0.77      0.76      0.76      1100


Accuracy: 0.764545454545
[[77  3  3  2  4  0  2  1  2  3  3]
 [ 1 74  1  0  2  0  6  8  0  2  6]
 [ 0  2 85  6  2  1  2  0  2  0  0]
 [ 0  0  3 91  0  2  0  1  3  0  0]
 [ 1  2  1  1 69  0  0  1  1 23  1]
 [ 0  1  4  3  0 85  0  0  3  1  3]
 [ 4

Что со знаками препинания, что без них -- все одинаково